# Obtener los datos del Parkinson

**APIS**

In [7]:
import pandas as pd
import requests

# URLs de la API
DATA_URL = "https://api.ourworldindata.org/v1/indicators/916408.data.json"
METADATA_URL = "https://api.ourworldindata.org/v1/indicators/916408.metadata.json"



**CARGA DE DATOS**

In [8]:
def cargar_datos():
   
    datos = requests.get(DATA_URL).json()  # Datos de casos de Parkinson
    metadata = requests.get(METADATA_URL).json()  # Información adicional (años y países)
    return datos, metadata


**PROCESAR DATOS**

In [9]:
def procesar_datos(datos, metadata):
    """Extrae la lista de valores, años disponibles y países desde los datos JSON."""
    
    # Extraer la lista de valores numéricos (casos de Parkinson)
    valores = datos["values"]
    
    # Extraer la lista de años disponibles
    años = []
    for item in metadata["dimensions"]["years"]["values"]:
        años.append(item["id"])
    
    # Extraer los países y sus nombres en un diccionario
    paises = {}
    for item in metadata["dimensions"]["entities"]["values"]:
        paises[item["id"]] = item["name"]
    
    return valores, años, paises


**CREAR EL DATAFRAME**

In [10]:
def crear_dataframe(valores, años, paises):
    
    num_años = len(años)
    num_valores = len(valores)

    if num_valores % num_años == 0:
        num_paises = num_valores // num_años
        print(f"Se encontraron datos de {num_paises} países.")

        # Construcción de la tabla con los datos
        datos_lista = []
        for i in range(num_valores):
            año = años[i % num_años]  # Seleccionar el año correspondiente
            id_pais = list(paises.keys())[i // num_años]  # Seleccionar el país correspondiente
            nombre_pais = paises[id_pais]  # Obtener el nombre del país

            # Crear un diccionario con los datos y agregarlo a la lista
            fila = {"Año": año, "Casos": valores[i], "País": nombre_pais}
            datos_lista.append(fila)

        # Convertir la lista de datos en un DataFrame
        df = pd.DataFrame(datos_lista)

    else:
       
        # Crear un DataFrame con solo los datos disponibles
        df = pd.DataFrame({"Año": años[:num_valores], "Casos": valores[:num_valores]})
        df["País"] = "Mundo"

    return df

In [11]:
datos, metadata = cargar_datos()  # Cargar datos desde la API
valores, años, paises = procesar_datos(datos, metadata)  # Extraer información
df = crear_dataframe(valores, años, paises)  # Crear DataFrame

# Mostrar los primeros resultados
print(df.head())


Se encontraron datos de 228 países.
    Año      Casos      País
0  1990  21.533830  Cambodia
1  1991  21.483840  Cambodia
2  1992  21.502138  Cambodia
3  1993  21.607151  Cambodia
4  1994  21.767658  Cambodia
